## 🧠 SimpleCNN
- Conv2D (3→32)
- MaxPool2D	(16x16x32)
- Conv2D (32→64)	(16x16x64)
- MaxPool2D	(8x8x64)
- Flatten	(4096)
- FC(4096→128)
- FC(128→10)

In [ ]:
# !pip install torchinfo

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import os
import json
import platform
import pkg_resources
from sklearn.metrics import precision_score, recall_score, f1_score
from torchinfo import summary

# GPU 설정
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 하이퍼파라미터 설정
BATCH_SIZE = 128
EPOCHS = 2
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-5
DROPOUT_RATE = 0.3
INPUT_SIZE = 3 * 32 * 32
NUM_CLASSES = 10
MODEL_PATH = './best_model.pth'
PATIENCE = 3
SPLIT_RATIO = 0.8

# CIFAR10 데이터셋의 평균 및 표준편차 (정규화 기준)
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

# 학습 데이터 증강 설정 (후반 학습용)
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# 테스트 데이터 변환 (데이터 증강 없이 정규화만 적용)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# 데이터셋 로드
full_train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root="./data", train=False, transform=transform_test, download=True)

# 학습 및 검증 데이터셋 분리
train_size = int(SPLIT_RATIO * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
val_dataset.dataset.transform = transform_test

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 모델 정의 (CNN)
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # [B, 3, 32, 32] → [B, 32, 32, 32]
            nn.ReLU(),
            nn.MaxPool2d(2),                             # → [B, 32, 16, 16]

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # → [B, 64, 16, 16]
            nn.ReLU(),
            nn.MaxPool2d(2),                             # → [B, 64, 8, 8]
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),                                # → [B, 64*8*8]
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(),
            nn.Dropout(DROPOUT_RATE),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# 모델 초기화
model = SimpleCNN().to(DEVICE)

# summary 호출
print(summary(model, input_size=(BATCH_SIZE, 3, 32, 32)))

# 모델 불러오기
def load_model():
    if os.path.exists(MODEL_PATH):
        model.load_state_dict(torch.load(MODEL_PATH))
        print('Model loaded from checkpoint.')
    else:
        print('No checkpoint found, training from scratch.')
    return model

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

best_val_acc = 0
patience_counter = 0

# 학습 함수 정의
def train():
    global best_val_acc, patience_counter
    model.train()
    for epoch in range(EPOCHS):

        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        scheduler.step()

        avg_loss = running_loss / len(train_loader)
        val_loss, val_acc = validate()

        print(f"Epoch [{epoch+1}/{EPOCHS}], Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:  # Early Stopping 체크
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), MODEL_PATH)
            print(f"New Best Model Saved! Validation Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print("Early stopping triggered.")
                break

def validate():
    model.eval()
    correct = 0
    total = 0
    val_loss_total = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss_total += loss.item()

            _, predicted = torch.max(outputs, 1) # 반환:(최댓값, 인덱스)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss_total / len(val_loader)
    val_acc = 100 * correct / total

    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"Validation - Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    return val_loss, val_acc


def test():
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f'Test Accuracy: {accuracy:.2f}%')
    print(f'Test Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')


# 최적의 하이퍼파라미터 정보와 환경 정보 저장 함수
def save_study_info(filename="best_study_info.json"):

    # Python 및 라이브러리 정보 가져오기
    python_version = platform.python_version()
    platform_info = platform.platform()
    installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}

    # 데이터셋 정보 수집
    dataset_name = 'CIFAR10'  # 현재 사용 중인 데이터셋 이름
    dataset_info = {
        "dataset_name": dataset_name,
        "train_size": len(train_dataset),
        "validation_size": len(val_dataset),
        "test_size": len(test_dataset),
        "image_size": (32, 32),  # CIFAR-10 이미지 크기
        "num_classes": NUM_CLASSES
    }

    # 모델 정보 수집
    model_info = {
        "model_name": model.__class__.__name__,
        "model_structure": str(model),
        "input_size": INPUT_SIZE,
        "num_classes": NUM_CLASSES
    }

    # 정보 저장
    data = {
        "python_version": python_version,
        "platform_info": platform_info,
        "installed_packages": installed_packages,
        "dataset_info": dataset_info,
        "model_info": model_info
    }

    with open(filename, "w") as f:
        json.dump(data, f, indent=4)

    print(f"Best study info saved to {filename}")


if __name__ == "__main__":
    train()
    # load_model()
    test()

    # 최적의 정보 저장
    save_study_info()

<ipython-input-1-325b337f7fb4>:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
100%|██████████| 170M/170M [00:03<00:00, 53.8MB/s]


Layer (type:depth-idx)                   Output Shape              Param #
SimpleCNN                                [128, 10]                 --
├─Sequential: 1-1                        [128, 64, 8, 8]           --
│    └─Conv2d: 2-1                       [128, 32, 32, 32]         896
│    └─ReLU: 2-2                         [128, 32, 32, 32]         --
│    └─MaxPool2d: 2-3                    [128, 32, 16, 16]         --
│    └─Conv2d: 2-4                       [128, 64, 16, 16]         18,496
│    └─ReLU: 2-5                         [128, 64, 16, 16]         --
│    └─MaxPool2d: 2-6                    [128, 64, 8, 8]           --
├─Sequential: 1-2                        [128, 10]                 --
│    └─Flatten: 2-7                      [128, 4096]               --
│    └─Linear: 2-8                       [128, 128]                524,416
│    └─ReLU: 2-9                         [128, 128]                --
│    └─Dropout: 2-10                     [128, 128]                --
│    

## 🧠 ResNet18
- ResidualBlock

```
        입력 x
          │
          ├───────┐
          │              │
          ▼              ▼
      Conv2D(3x3)     (shortcut)
          ↓              │
      BatchNorm           │
          ↓              │
         ReLU             │
          ↓              │
      Conv2D(3x3)         │
          ↓              │
       BatchNorm          │
          ↓              │
      처리된 결과 out ◄─-┘ ← identity (x 또는 변형된 x)
             │
             ▼
         out + identity
             │
             ▼
            ReLU
             │
             ▼
         최종 출력
```

- CustomResNet18


In [ ]:
# !pip install torchinfo

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import os
import json
import platform
import pkg_resources
from sklearn.metrics import precision_score, recall_score, f1_score
from torchinfo import summary

# GPU 설정
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 하이퍼파라미터 설정
BATCH_SIZE = 128
EPOCHS = 2
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-5
DROPOUT_RATE = 0.3
INPUT_SIZE = 3 * 32 * 32
NUM_CLASSES = 10
MODEL_PATH = './best_model.pth'
PATIENCE = 3
SPLIT_RATIO = 0.8

# CIFAR10 데이터셋의 평균 및 표준편차 (정규화 기준)
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

# 학습 데이터 증강 설정 (후반 학습용)
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# 테스트 데이터 변환 (데이터 증강 없이 정규화만 적용)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# 데이터셋 로드
full_train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root="./data", train=False, transform=transform_test, download=True)

# 학습 및 검증 데이터셋 분리
train_size = int(SPLIT_RATIO * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
val_dataset.dataset.transform = transform_test

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 모델 정의 (ResNet18)
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # 스킵 커넥션 (차원 맞춰주는 역할)
        if in_channels != out_channels or stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Identity()

        self.relu = nn.ReLU()

    def forward(self, x):
        identity = self.shortcut(x)

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        out = self.relu(out)

        return out

class CustomResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        # Residual Blocks (2개씩 반복)
        self.layer1 = nn.Sequential(
            ResidualBlock(64, 64),
            ResidualBlock(64, 64)
        )
        self.layer2 = nn.Sequential(
            ResidualBlock(64, 128, stride=2),
            ResidualBlock(128, 128)
        )
        self.layer3 = nn.Sequential(
            ResidualBlock(128, 256, stride=2),
            ResidualBlock(256, 256)
        )
        self.layer4 = nn.Sequential(
            ResidualBlock(256, 512, stride=2),
            ResidualBlock(512, 512)
        )

        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))  # 초기 conv

        x = self.layer1(x)  # [B, 64, 32, 32]
        x = self.layer2(x)  # [B, 128, 16, 16]
        x = self.layer3(x)  # [B, 256, 8, 8]
        x = self.layer4(x)  # [B, 512, 4, 4]

        x = self.global_pool(x)  # [B, 512, 1, 1]
        x = x.view(x.size(0), -1)  # [B, 512]
        x = self.fc(x)  # [B, 10]

        return x

# 모델 초기화
model = CustomResNet18(num_classes=NUM_CLASSES).to(DEVICE)

# summary 호출
print(summary(model, input_size=(BATCH_SIZE, 3, 32, 32)))

# 모델 불러오기
def load_model():
    if os.path.exists(MODEL_PATH):
        model.load_state_dict(torch.load(MODEL_PATH))
        print('Model loaded from checkpoint.')
    else:
        print('No checkpoint found, training from scratch.')
    return model

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

best_val_acc = 0
patience_counter = 0

# 학습 함수 정의
def train():
    global best_val_acc, patience_counter
    model.train()
    for epoch in range(EPOCHS):

        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        scheduler.step()

        avg_loss = running_loss / len(train_loader)
        val_loss, val_acc = validate()

        print(f"Epoch [{epoch+1}/{EPOCHS}], Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:  # Early Stopping 체크
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), MODEL_PATH)
            print(f"New Best Model Saved! Validation Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print("Early stopping triggered.")
                break

def validate():
    model.eval()
    correct = 0
    total = 0
    val_loss_total = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss_total += loss.item()

            _, predicted = torch.max(outputs, 1) # 반환:(최댓값, 인덱스)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss_total / len(val_loader)
    val_acc = 100 * correct / total

    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"Validation - Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    return val_loss, val_acc


def test():
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f'Test Accuracy: {accuracy:.2f}%')
    print(f'Test Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')


# 최적의 하이퍼파라미터 정보와 환경 정보 저장 함수
def save_study_info(filename="best_study_info.json"):

    # Python 및 라이브러리 정보 가져오기
    python_version = platform.python_version()
    platform_info = platform.platform()
    installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}

    # 데이터셋 정보 수집
    dataset_name = 'CIFAR10'  # 현재 사용 중인 데이터셋 이름
    dataset_info = {
        "dataset_name": dataset_name,
        "train_size": len(train_dataset),
        "validation_size": len(val_dataset),
        "test_size": len(test_dataset),
        "image_size": (32, 32),  # CIFAR-10 이미지 크기
        "num_classes": NUM_CLASSES
    }

    # 모델 정보 수집
    model_info = {
        "model_name": model.__class__.__name__,
        "model_structure": str(model),
        "input_size": INPUT_SIZE,
        "num_classes": NUM_CLASSES
    }

    # 정보 저장
    data = {
        "python_version": python_version,
        "platform_info": platform_info,
        "installed_packages": installed_packages,
        "dataset_info": dataset_info,
        "model_info": model_info
    }

    with open(filename, "w") as f:
        json.dump(data, f, indent=4)

    print(f"Best study info saved to {filename}")


if __name__ == "__main__":
    train()
    # load_model()
    test()

    # 최적의 정보 저장
    save_study_info()

Layer (type:depth-idx)                   Output Shape              Param #
CustomResNet18                           [128, 10]                 --
├─Conv2d: 1-1                            [128, 64, 32, 32]         1,728
├─BatchNorm2d: 1-2                       [128, 64, 32, 32]         128
├─ReLU: 1-3                              [128, 64, 32, 32]         --
├─Sequential: 1-4                        [128, 64, 32, 32]         --
│    └─ResidualBlock: 2-1                [128, 64, 32, 32]         --
│    │    └─Identity: 3-1                [128, 64, 32, 32]         --
│    │    └─Conv2d: 3-2                  [128, 64, 32, 32]         36,864
│    │    └─BatchNorm2d: 3-3             [128, 64, 32, 32]         128
│    │    └─ReLU: 3-4                    [128, 64, 32, 32]         --
│    │    └─Conv2d: 3-5                  [128, 64, 32, 32]         36,864
│    │    └─BatchNorm2d: 3-6             [128, 64, 32, 32]         128
│    │    └─ReLU: 3-7                    [128, 64, 32, 32]         --
│

## 🧠 Pretrained ResNet18
- 🔧 필요한 import 추가

```python
from torchvision.models import resnet18
```

- 🧠 모델 생성 함수 만들기 (사전학습 가중치 사용)

```python
def build_pretrained_resnet18(num_classes=10):
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    # 선택: CIFAR에 맞게 conv1 수정 (안 해도 동작은 함)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()

    return model.to(DEVICE)
```

- 🧠 전이학습 옵션

```python
# 1. 모델 정의
model = resnet18(pretrained=True)

# 2. 출력 레이어 수정 (1000 → 10)
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

# 3. 파라미터 학습 여부 설정
for param in model.parameters():
    param.requires_grad = False  # 전체 동결 (freeze)

for param in model.fc.parameters():
    param.requires_grad = True   # fc만 학습 가능

# 4. 모델 디바이스로 이동
model = model.to(DEVICE)

```

In [ ]:
# !pip install torchinfo

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import os
import json
import platform
import pkg_resources
from sklearn.metrics import precision_score, recall_score, f1_score
from torchinfo import summary
from torchvision.models import resnet18

# GPU 설정
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 하이퍼파라미터 설정
BATCH_SIZE = 128
EPOCHS = 2
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-5
DROPOUT_RATE = 0.3
INPUT_SIZE = 3 * 32 * 32
NUM_CLASSES = 10
MODEL_PATH = './best_model.pth'
PATIENCE = 3
SPLIT_RATIO = 0.8

# CIFAR10 데이터셋의 평균 및 표준편차 (정규화 기준)
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

# 학습 데이터 증강 설정 (후반 학습용)
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# 테스트 데이터 변환 (데이터 증강 없이 정규화만 적용)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
])

# 데이터셋 로드
full_train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root="./data", train=False, transform=transform_test, download=True)

# 학습 및 검증 데이터셋 분리
train_size = int(SPLIT_RATIO * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])
val_dataset.dataset.transform = transform_test

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 모델 불러오기 (ResNet18)
def build_pretrained_resnet18(num_classes=10):
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    # 선택: CIFAR에 맞게 conv1 수정 (안 해도 동작은 함)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()

    # 파라미터 학습 여부 설정
    for param in model.parameters():
        param.requires_grad = False  # 전체 동결 (freeze)

    for param in model.fc.parameters():
        param.requires_grad = True   # fc만 학습 가능

    return model.to(DEVICE)

# 모델 초기화
model = build_pretrained_resnet18(num_classes=NUM_CLASSES)

# summary 호출
print(summary(model, input_size=(BATCH_SIZE, 3, 32, 32)))

# 모델 불러오기
def load_model():
    if os.path.exists(MODEL_PATH):
        model.load_state_dict(torch.load(MODEL_PATH))
        print('Model loaded from checkpoint.')
    else:
        print('No checkpoint found, training from scratch.')
    return model

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

best_val_acc = 0
patience_counter = 0

# 학습 함수 정의
def train():
    global best_val_acc, patience_counter
    model.train()
    for epoch in range(EPOCHS):

        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        scheduler.step()

        avg_loss = running_loss / len(train_loader)
        val_loss, val_acc = validate()

        print(f"Epoch [{epoch+1}/{EPOCHS}], Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:  # Early Stopping 체크
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), MODEL_PATH)
            print(f"New Best Model Saved! Validation Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print("Early stopping triggered.")
                break

def validate():
    model.eval()
    correct = 0
    total = 0
    val_loss_total = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss_total += loss.item()

            _, predicted = torch.max(outputs, 1) # 반환:(최댓값, 인덱스)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss_total / len(val_loader)
    val_acc = 100 * correct / total

    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"Validation - Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

    return val_loss, val_acc


def test():
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f'Test Accuracy: {accuracy:.2f}%')
    print(f'Test Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')


# 최적의 하이퍼파라미터 정보와 환경 정보 저장 함수
def save_study_info(filename="best_study_info.json"):

    # Python 및 라이브러리 정보 가져오기
    python_version = platform.python_version()
    platform_info = platform.platform()
    installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}

    # 데이터셋 정보 수집
    dataset_name = 'CIFAR10'  # 현재 사용 중인 데이터셋 이름
    dataset_info = {
        "dataset_name": dataset_name,
        "train_size": len(train_dataset),
        "validation_size": len(val_dataset),
        "test_size": len(test_dataset),
        "image_size": (32, 32),  # CIFAR-10 이미지 크기
        "num_classes": NUM_CLASSES
    }

    # 모델 정보 수집
    model_info = {
        "model_name": model.__class__.__name__,
        "model_structure": str(model),
        "input_size": INPUT_SIZE,
        "num_classes": NUM_CLASSES
    }

    # 정보 저장
    data = {
        "python_version": python_version,
        "platform_info": platform_info,
        "installed_packages": installed_packages,
        "dataset_info": dataset_info,
        "model_info": model_info
    }

    with open(filename, "w") as f:
        json.dump(data, f, indent=4)

    print(f"Best study info saved to {filename}")


if __name__ == "__main__":
    train()
    # load_model()
    test()

    # 최적의 정보 저장
    save_study_info()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 90.3MB/s]


Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [128, 10]                 --
├─Conv2d: 1-1                            [128, 64, 32, 32]         (1,728)
├─BatchNorm2d: 1-2                       [128, 64, 32, 32]         (128)
├─ReLU: 1-3                              [128, 64, 32, 32]         --
├─Identity: 1-4                          [128, 64, 32, 32]         --
├─Sequential: 1-5                        [128, 64, 32, 32]         --
│    └─BasicBlock: 2-1                   [128, 64, 32, 32]         --
│    │    └─Conv2d: 3-1                  [128, 64, 32, 32]         (36,864)
│    │    └─BatchNorm2d: 3-2             [128, 64, 32, 32]         (128)
│    │    └─ReLU: 3-3                    [128, 64, 32, 32]         --
│    │    └─Conv2d: 3-4                  [128, 64, 32, 32]         (36,864)
│    │    └─BatchNorm2d: 3-5             [128, 64, 32, 32]         (128)
│    │    └─ReLU: 3-6                    [128, 64, 32, 32] 